In [8]:
# imports
import numpy as np
import cv2 as cv
import glob


# termination criteria
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)

# Real world coordinates of circular grid
obj3d = np.zeros((44, 3), np.float32)
# As the actual circle size is not required,
# the z-coordinate is zero and the x and y coordinates are random numbers.
a = [0, 36, 72, 108, 144, 180, 216, 252, 288, 324, 360]
b = [0, 72, 144, 216, 36, 108, 180, 252]
for i in range(0, 44):
    obj3d[i] = (a[i // 4], (b[i % 8]), 0)
    # print(objp[i])

# Vector to store 3D points
obj_points = []
# Vector to store 2D points
img_points = []

# Extracting path of individual image stored in a given directory
images = glob.glob('./calibratecamera/*.png')
for f in images:
    # Loading image
    img = cv.imread(f)
    # Conversion to grayscale image
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

    # To find the position of circles in the grid pattern
    ret, corners = cv.findCirclesGrid(
        gray, (4, 11), None, flags=cv.CALIB_CB_ASYMMETRIC_GRID)

    # If true is returned, 
    # then 3D and 2D vector points are updated and corner is drawn on image
    if ret == True:
        obj_points.append(obj3d)

        corners2 = cv.cornerSubPix(gray, corners, (11, 11), (-1, -1), criteria)
        # In case of circular grids, 
        # the cornerSubPix() is not always needed, so alternative method is:
        # corners2 = corners
        img_points.append(corners2)

        # Drawing the corners, saving and displaying the image
        cv.drawChessboardCorners(img, (4, 11), corners2, ret)
        cv.imwrite('output.jpg', img) #To save corner-drawn image
        cv.imshow('img', img)
        cv.waitKey(0)
cv.destroyAllWindows()

"""Camera calibration: 
Passing the value of known 3D points (obj points) and the corresponding pixel coordinates 
of the detected corners (img points)"""
ret, camera_mat, distortion, rotation_vecs, translation_vecs = cv.calibrateCamera(
    obj_points, img_points, gray.shape[::-1], None, None)

print("Error in projection : \n", ret)
print("\nCamera matrix : \n", camera_mat)
print("\nDistortion coefficients : \n", distortion)
print("\nRotation vector : \n", rotation_vecs)
print("\nTranslation vector : \n", translation_vecs)

Error in projection : 
 0.28397138993192417

Camera matrix : 
 [[ 2.98018946e+03  0.00000000e+00 -2.07790644e+02]
 [ 0.00000000e+00  2.98680309e+03  5.80328416e+02]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00]]

Distortion coefficients : 
 [[-1.38990879e+00  1.28121501e+01 -1.76642504e-02  4.92392900e-02
  -6.65051660e+01]]

Rotation vector : 
 (array([[1.98945525],
       [2.57035288],
       [0.00544978]]), array([[-2.141599  ],
       [-1.96709247],
       [ 0.1071076 ]]), array([[-2.30654667],
       [-1.91925376],
       [-0.09466172]]), array([[-2.14337584],
       [-1.80887689],
       [-0.09635144]]), array([[-1.89996746],
       [-1.89926953],
       [ 0.33507546]]))

Translation vector : 
 (array([[ 465.62345471],
       [-601.35082796],
       [3539.69468578]]), array([[ 549.35638975],
       [-550.85288875],
       [3547.04309867]]), array([[ 286.79704145],
       [-575.92278998],
       [3538.10081969]]), array([[ 379.75106914],
       [-604.7774144 ],
       [3469.4